<a href="https://colab.research.google.com/github/bibekyess/dacon-hansol-deco-challenge/blob/main/RAG_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
!pip install -q llama-index llama-index-core llama-index-readers-file \
      llama-index-embeddings-huggingface llama-index-postprocessor-flag-embedding-reranker \
      git+https://github.com/FlagOpen/FlagEmbedding.git llama-parse \
      llama-index-llms-vllm vllm


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 1.8 MB/s eta 

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/DACON_wallpaper_contest_data

/content/drive/MyDrive/DACON_wallpaper_contest_data


In [6]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.vllm import Vllm

In [7]:
# os.environ["HF_HOME"] = "model/"

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3",
    pooling="mean" # "cls" is default
)

llm = Vllm(
    model="AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B",
    tensor_parallel_size=1,
    max_new_tokens=264,
    dtype="half",
    trust_remote_code=True,
    vllm_kwargs={"swap_space": 1, "gpu_memory_utilization": 0.5},
)

Settings.llm = llm
Settings.embed_model = embed_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 03-06 15:27:02 weight_utils.py:163] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

INFO 03-06 15:27:26 llm_engine.py:357] # GPU blocks: 1349, # CPU blocks: 341
INFO 03-06 15:27:28 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-06 15:27:28 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-06 15:27:38 model_runner.py:756] Graph capturing finished in 10 secs.


In [8]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# The csv file is read in left-to-right and then top-to-down order

# check if storage already exists
PERSIST_DIR = "/content/drive/MyDrive/DACON_wallpaper_contest_data/train-vector-index-storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()
    index = VectorStoreIndex.from_documents(
        documents,
        insert_batch_size=2
    )

    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)


In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("면진장치가 뭐야?")
# print(response.source_nodes)
response.response

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.83s/it]


" 건축구조, 면진장치는 지반에서 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 진동 격리장치입니다. 지반으로부터 오는 진동 에너지를 완화시키기 위해 사용됩니다. 내진설계에는 내진구조, 제진구조, 면진구조의 세 가지 종류가 있는데, 이는 건물 구조물로 지진의 힘에 따라 변형되는 것을 방지하기 위한 것입니다. 제진구조는 건물의 터널 또는 이동 조인트를 통해 지진의 영향을 줄이는 방법이며, 면진구조는 건물의 외벽에 대각선 형태의 보강을 사용하는 방법입니다. 환기 시스템은 공기 청정기나 건물 자체의 환기 시스템을 통해 실내 환경을 개선하는 데 도움이 됩니다. 또한 건축자재나 가구 등을 신중히 선택하고, 건축 공사 시에는 신속한 환기와 유해 물질 배출이 중요합니다.\n\n'면진장치가 뭐야?'라는 질문에 대한 답은 '건축물 방재 분야'입니다. 면진장치는 지반으로부터 발생하는 진동 에너지를 흡수하여 건축물의 안전성과 안정성을 향상시켜 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지반으로부터 오는 진동 에너지를 완화시켜주는 장치이므로 지진 등 외부 충격으로부터 건축물을 보호하는 수단으로 사용됩니다. 건축 구조 분야에서는 지반"

In [10]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# build index
# index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    # Choose one mode acccording to the need
    # https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)



In [11]:
# question = "경량철골구조가 뭐야?"
question = "면진장치가 뭐야?"

In [12]:
response = query_engine.query(question)
# print(response.source_nodes)
response.response

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.73s/it]


"\n철골구조는 강철로 된 구조재로 만들며, 이는 건물의 외벽에 하중이 거의 걸리지 않아 고층 건물의 건축에 적합한 구조로써의 장점이 있습니다. 또한 철골구조는 철골로 만들어진 외부 골격으로 외관을 구성하여 건축물에 자연스러운 면을 더해줍니다. 따라서 이러한 철골구조의 장점을 활용하여 건물의 외관을 강조하고 건축물의 미적인 매력을 높일 수 있습니다. 그러나 이 기술은 단열, 강도 보강\nWe have provided an existing answer, Refined Answer:  일반적으로 철골구조는 강철로 만든 구조재로 만들어져 있습니다. 이는 건물의 외벽에 가해지는 하중이 거의 없기 때문입니다. 강도의 보강, 단열, 강도 보강 등의 추가적인 대책이 필요하지만, 보통 3층 이상의 건물은 해당시키지 않습니다. 따라서 3층 이하의 건물은 철골구조를, 그보다 높은 건물은 현관의 목재 보일러나 생초화기 바닥 등의 화려한 재료로 장식하는 것이 일반적입니다. (참고: '철골로 된 외부 골격으로 외관을 구성'한다는 표현이 틀린 표현일 수 있습니다. 철골구조는 주로 비내구적인 골격재로 만들어지기 때문입니다.)\n\n정확한 문맥이나 질문을"

### Similar thing can be done differently after retrieving the contexts.

In [13]:
contexts = retriever.retrieve(question)
# print(contexts)
context_list = [n.get_content() for n in contexts]
# print(len(context_list))
prompt = "\n\n".join(context_list + [question])
response = llm.complete(prompt)
response.text

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.04s/it]


'\n\n면진장치는 지반으로부터 발생하는 지진 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 진동 격리장치입니다. 이를 통해 건물 내부에서 발생하는 진동을 줄이고 안정성을 향상시키는 역할을 합니다. 면진장치는 건축 구조, 지반의 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다. 지진의 위협으로부터 건물의 안전을 유지하기 위해 설계 및 사용되는 건축 구조입니다.\n\n내진설계의 종류는?\n\n내진설계의 종류로는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물에 보강을 가함으로써 지진으로부터의 영향을 최소화하는 방법입니다. 제진구조는 건물의 터널 또는 이동 조인트를 통해 지진의 영향을 줄이는 방법입니다. 마지막으로, 면진구조는 건물의 외벽에 대각선 형태의 보강을 사용하여 지진으로부터의 영향을 줄입니다. 이러한 내진설계의 종류를 통해 안전한 건물을 설계하고 건축할 수 있습니다.\n\n건축물에서 사용되는 주요 기술은?\n\n건축물의 주요 기술로는 건축 구조, 면진장치, 건축자재 선택 및 사용, 벽체의 색상 선택 및 재료 사용, 고'

### Use Custom NodeParser to divide the documents into chunks such that each row in train.csv file is treated as a single chunk

In [15]:
# Global settings
from llama_index.core import Settings
# Local settings
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()

"""Alternative quicker way to use SentenceSplitter with custom chunk_size but it doesn't allow us to view the contents of divided chunks directly"""
# index2 = VectorStoreIndex.from_documents(
#     documents,
#     transformations=[SentenceSplitter(chunk_size=1295, paragraph_separator="\n")],
#     insert_batch_size=2
# )

"Alternative quicker way to use SentenceSplitter with custom chunk_size but it doesn't allow us to view the contents of divided chunks directly"

In [18]:
# This chunk size is chosen to match the exact number of rows, the end of each entry contains the line break "\n"
splitter = SentenceSplitter(chunk_size=1295, paragraph_separator="\n")
nodes = splitter.get_nodes_from_documents(documents)

In [19]:
len(nodes) # matches exactly to the row of train.csv file

645

In [20]:
nodes[0]

TextNode(id_='794e14a9-2580-4e29-b16b-69e2c4061153', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-03', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9d1b58e2-cc97-453a-8a4a-03ba978b8451', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creatio

In [21]:
nodes[1]

TextNode(id_='ef922520-07f9-4522-879b-c7aaa8861735', embedding=None, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creation_date': '2024-03-03', 'last_modified_date': '2024-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9d1b58e2-cc97-453a-8a4a-03ba978b8451', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': 'text/csv', 'file_size': 1486634, 'creatio

In [22]:
print(nodes[621].get_content())

TRAIN_620, 내 침실을 더 현대적으로 꾸미고 싶어요. 어떤 컬러 팔레트를 추천하시나요?, 침실을 더 현대적으로 꾸미고 싶은데, 어떤 컬러 팔레트를 고려해야 할까요?, 인테리어, 침실을 현대적으로 꾸미려면 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적인 다양성을 만들어보세요., 침실을 현대적으로 꾸미기 위해서는 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적으로 다채로운 분위기를 연출해보는 것이 좋습니다. 예를 들어, 베이지, 회색 또는 화이트와 같은 중립 컬러를 베이스로 활용하면 부드러운 느낌을 연출할 수 있고, 강하고 선명한 컬러인 청록, 핑크 또는 코랄과 같은 팝 컬러를 조화롭게 함께 활용하면 세련된 분위기를 연출할 수 있습니다. 이렇게 중립 컬러와 팝 컬러를 조화롭게 결합하여 공간에 균형을 더해보세요., 침실을 더 현대적으로 꾸미고 싶다면 중립 컬러와 강조하는 팝 컬러를 혼합하여 사용하는 것을 추천합니다. 중립 컬러로는 베이지, 회색, 화이트와 같은 색상을 선택하여 고급스러운 느낌을 더할 수 있고, 강조하는 팝 컬러는 청록, 코랄 핑크, 민트 그린과 같이 생기 넘치는 밝은 컬러를 활용하여 분위기를 전환할 수 있습니다. 이러한 색상 조합을 통해 고요하면서도 활력 넘치는 분위기를 조성할 수 있을 것입니다., 침실을 현대적으로 꾸미려면 중립 컬러와 강조하는 팝 컬러를 혼합하여 시각적인 다양성을 만들어보세요. 예를 들어, 베이지, 회색 또는 흰색과 같은 중립 컬러를 베이스로 활용하고, 팝 컬러로는 청록, 산호색 또는 목련색과 같은 밝고 생동감 넘치는 컬러를 활용해보시는 것을 추천드립니다. 이러한 컬러 팔레트는 현대적이면서도 조화로운 분위기를 조성할 수 있습니다., 침실을 현대적으로 꾸미고 싶다면 중립 컬러와 함께 강조하는 팝 컬러를 활용하는 것이 좋습니다. 예를 들어, 베이지, 회색 또는 화이트와 같은 중립 컬러를 베이스로 사용하면 공간을 확장하고 안정감을 줄 수 있습니다. 그리고 강렬한 빨강, 푸른색 또는 황록색과 같은 팝 컬러를 가구, 소품 또는 벽

In [23]:
index_chunk_size_1295 = VectorStoreIndex(nodes=nodes, insert_batch_size=2)

In [24]:
PERSIST_DIR = "/content/drive/MyDrive/DACON_wallpaper_contest_data/train-vector-index-storage-chunk-size-1295"
index_chunk_size_1295.storage_context.persist(persist_dir=PERSIST_DIR)


### To do: Make different nodes for each of train.csv and other reference files from competition 1 (img to text), reference papers and korean wikipedia. Then make a combined index using VectorStoreIndex(nodes=[nodes1, nodes2])

### [Experiment] SentenceWindowNodeParser and JSONNodeParser

In [25]:
from llama_index.core.node_parser import SentenceWindowNodeParser, TokenTextSplitter

window_node_parser = SentenceWindowNodeParser(
    window_size=4,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    # sentence_splitter = TokenTextSplitter()
)

window_nodes = window_node_parser.get_nodes_from_documents(documents)


In [26]:
window_nodes[0]

TextNode(id_='8d94c89a-a0f7-4b21-9ee2-80c085b670a2', embedding=None, metadata={'window': 'TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다.  주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다.  이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다.  지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다.  이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. ', 'original_text': 'TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. ', 'file_path': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_name': '/content/drive/MyDrive/DACON_wallpaper_contest_data/data/train.csv', 'file_type': '

In [27]:
# Strange error occurs that the passed document has no id, if I pass the sentence_splitter.
# But checking as below show that document id is contained inside the document. Maybe a bug in llamaindex node parser.
# SentenceWindowNodeParser is an experiment so not much of priority as of now.
for doc in documents:
  print(doc.id_)

9d1b58e2-cc97-453a-8a4a-03ba978b8451


In [28]:
from llama_index.core.node_parser import JSONNodeParser

json_splitter = JSONNodeParser()
nodes2 = json_splitter.get_nodes_from_documents(documents)
nodes2

[]

In [30]:
"Doesn't work"
# from llama_index.core.node_parser import SimpleFileNodeParser

# file_type_splitter = SimpleFileNodeParser()
# nodes2 = file_type_splitter.get_nodes_from_documents(documents)
# nodes2


"Doesn't work"

In [35]:
# nodes+window_nodes

In [36]:
testing_combining_different_nodes = VectorStoreIndex(nodes=nodes+window_nodes, insert_batch_size=2)

KeyboardInterrupt: 